In [ ]:
#Code to get the load_csv_database function.
#We pass this function as context so there's no need to include it when we parse the notebook
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..", "..")))
from spider2_utils import load_csv_database

-setup-

In [ ]:
import pandas as pd
_database = load_csv_database("California_Traffic_Collision", rows_limit=-1)
collisions = _database["collisions"]

# Question
In which year were the two most common causes of traffic accidents different from those in other years?

# Step 1: Calculate Category Totals
Aggregate the total number of cases for each category and year.

In [ ]:
category_totals = collisions.copy()
category_totals['Year'] = pd.to_datetime(category_totals['collision_date']).dt.year
category_totals = category_totals.groupby(['Year', 'pcf_violation_category']).agg(
    Subtotal=('case_id', 'count')
).reset_index()

# Step 2: Find the Top 2 Causes for Each Year
For each year, identify the two most common primary collision factor violation categories.

In [ ]:
top2_causes_per_year = category_totals.sort_values(['Year', 'Subtotal'], ascending=[True, False]) \
    .groupby('Year').head(2)
top2_causes_per_year = top2_causes_per_year.groupby('Year')['pcf_violation_category'].apply(list).reset_index()
top2_causes_per_year.columns = ['Year', 'Top2Causes']

# Step 3: Compare Top 2 Causes Across Years
Find the years where the top 2 causes are different from those in all other years.

In [ ]:
# Find years with unique top 2 causes
from collections import Counter
cause_counts = Counter(tuple(sorted(causes)) for causes in top2_causes_per_year['Top2Causes'])
top2_causes_per_year['IsUnique'] = top2_causes_per_year['Top2Causes'].apply(lambda x: cause_counts[tuple(sorted(x))] == 1)
unique_years = top2_causes_per_year[top2_causes_per_year['IsUnique']]
final_result = unique_years[['Year']]